In [ ]:
# Imports
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Scipy center of mass function
from scipy.ndimage.measurements import center_of_mass as CoM

In [2]:
# Set the location of the LArCV 512x512 data
data_root = "/media/disk1/kai/larcv_png_data/larcv_png_512/larcv_png_512/"
image_data = []

In [ ]:
# Load the 512x512 data
print("Loading images from {}".format(data_root))

for image_path in os.listdir(data_root):
    in_path = os.path.join(data_root, image_path)
    image_data.append(cv.imread(in_path, 0))

print("Finished loading {} images".format(len(image_data)))

Once the LArCV 512 image data is loaded, cropping procedure can be run any number of times

In [ ]:
# Set the desired crop size and set the save location for the new image crops
crop_dim = 256
im_crops = []
save_path = "/media/hdd1/kai/particle_generator/larcv_data/larcv_png_{}/larcv_png_{}/".format(crop_dim, crop_dim)

In [1]:
# Set up variables for logging centroids
centroids_x = []; centroids_y = []

### Compute set of cropped images

In [ ]:
imsize = 512
for idx, image in enumerate(image_data):

    # Check that the image is at least 1% filled
    pix_check = np.where(image > 0, 1, 0)
    pix_percentage = (np.sum(pix_check) / pix_check.flatten().shape[0]) > 0.01
    if not pix_percentage:
        del image
        continue
    del pix_check

    # Get the weighted centroid position
    com = CoM(image)
    
    # check if computation returned NaN -- happens when centroid location is non-sensical
    if com[0] != com[0]:
        del image
        continue

    # Track centroids
    centroids_x.append(int(com[0]))
    centroids_y.append(int(com[1]))
    
    # Ensure that the centroid location in within acceptable bounds from the edge
    check_x = (centroid_x[-1] > crop_dim) and ((imsize - centroid_x[-1]) > crop_dim)
    check_y = (centroid_y[-1] > crop_dim) and ((imsize - centroid_y[-1]) > crop_dim)
    
    if check_x and check_y:
        x1 = centroids_x[-1] - crop_dim; x2 = centroids_x[-1] + crop_dim
        y1 = centroids_y[-1] - crop_dim; y2 = centroids_y[-1] + crop_dim
        im_crops.append(image[x1:x2, y1:y2])
    else:
        del image
        continue

In [ ]:
print("-"*60)
print("Generated {} images at [{}x{}] resolution".format(len(im_crops), crop_dim, crop_dim))
print("This represents ~{} percent of the original dataset".format( round(len(im_crops)/len(image_data),2) ) )
print("-"*60)

### Grab sample of newly cropped images

In [ ]:
fig = plt.figure()
n_row = 4; n_col = 4

for i in range(1, nrow*n_col + 1):
    fig.add_subplot(n_row, n_col, i)
    plt.imshow(im_crops[i])

### Save newly cropped images to specified save_path

In [ ]:
# Check that save_path exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

for idx, crop in enumerate(im_crops):
    im_name = "larcv_{}_{}.png".format(crop_dim, idx)
    out_path = os.path.join(save_path, im_name)
    cv.imwrite(out_path, crop)